# Import & Preprocessing (for raw TF-IDF system/model)

In [24]:
# import
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# load dataset
df = pd.read_csv('scraping/halodocnew_1.csv')
print(df.shape)

# check dataset
df.head()

(9979, 5)


,url,title,title_count,content,content_count
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"Ibu, Perhatikan 5 Hal Ini saat Memilih Car Se...",10,"Bagi orang tua yang memiliki kendaraan, memili...",540
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"Catat, Ini 6 Tips Menyimpan Beras agar Tidak ...",9,Sebagai makanan pokok beras harus disimpan den...,564
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"Waspada, Ini 6 Bahaya Asap Rokok bagi Kesehat...",9,Merokok tidak hanya berbahaya untuk dirimu sen...,649
3,https://www.halodoc.com/sering-terkena-polusi-...,"Sering Terkena Polusi, Ketahui Berbagi Cara B...",8,"Sehari-hari, manusia terpapar polusi dalam jum...",523
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,Ini Risiko dan Manfaat Implan Payudara yang P...,9,Implan payudara adalah prosedur bedah kosmetik...,547


In [25]:
# remove 'content' that contains 'halodoc' (dianggap iklan/promosi, tidak terlalu berpengaruh ke konteks dokumen/teks)
df['content'] = df['content'].apply(lambda x: '.'.join([s for s in x.split('.') if 'halodoc' not in s.lower()]))

In [26]:
# make information retrieval system
# Query = df['title']
# Document = df['content']

# Preprocessing
# 1. Case Folding
# 2. Tokenizing
# 3. Filtering
# 4. Stemming

# Case Folding
df['title'] = df['title'].str.lower()
df['content'] = df['content'].str.lower()
df.head()


,url,title,title_count,content,content_count
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"ibu, perhatikan 5 hal ini saat memilih car se...",10,"bagi orang tua yang memiliki kendaraan, memili...",540
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"catat, ini 6 tips menyimpan beras agar tidak ...",9,sebagai makanan pokok beras harus disimpan den...,564
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"waspada, ini 6 bahaya asap rokok bagi kesehat...",9,merokok tidak hanya berbahaya untuk dirimu sen...,649
3,https://www.halodoc.com/sering-terkena-polusi-...,"sering terkena polusi, ketahui berbagi cara b...",8,"sehari-hari, manusia terpapar polusi dalam jum...",523
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,ini risiko dan manfaat implan payudara yang p...,9,implan payudara adalah prosedur bedah kosmetik...,547


In [27]:
nltk.download('punkt')

# Tokenize the title column
df['title_tokens'] = df['title'].apply(nltk.word_tokenize)
df['title_count'] = df['title_tokens'].apply(len)

# Tokenize the content column
df['content_tokens'] = df['content'].apply(nltk.word_tokenize)
df['content_count'] = df['content_tokens'].apply(len)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Magezter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
df.head()

,url,title,title_count,content,content_count,title_tokens,content_tokens
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"ibu, perhatikan 5 hal ini saat memilih car se...",11,"bagi orang tua yang memiliki kendaraan, memili...",575,"[ibu, ,, perhatikan, 5, hal, ini, saat, memili...","[bagi, orang, tua, yang, memiliki, kendaraan, ..."
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"catat, ini 6 tips menyimpan beras agar tidak ...",10,sebagai makanan pokok beras harus disimpan den...,617,"[catat, ,, ini, 6, tips, menyimpan, beras, aga...","[sebagai, makanan, pokok, beras, harus, disimp..."
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"waspada, ini 6 bahaya asap rokok bagi kesehat...",10,merokok tidak hanya berbahaya untuk dirimu sen...,708,"[waspada, ,, ini, 6, bahaya, asap, rokok, bagi...","[merokok, tidak, hanya, berbahaya, untuk, diri..."
3,https://www.halodoc.com/sering-terkena-polusi-...,"sering terkena polusi, ketahui berbagi cara b...",9,"sehari-hari, manusia terpapar polusi dalam jum...",569,"[sering, terkena, polusi, ,, ketahui, berbagi,...","[sehari-hari, ,, manusia, terpapar, polusi, da..."
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,ini risiko dan manfaat implan payudara yang p...,9,implan payudara adalah prosedur bedah kosmetik...,613,"[ini, risiko, dan, manfaat, implan, payudara, ...","[implan, payudara, adalah, prosedur, bedah, ko..."


In [29]:
# Filtering
# Remove stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

# Remove stopwords from title
df['title_tokens_stopwords'] = df['title_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df['title_tokens_stopwords_count'] = df['title_tokens_stopwords'].apply(len)

# Remove stopwords from content
df['content_tokens_stopwords'] = df['content_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df['content_tokens_stopwords_count'] = df['content_tokens_stopwords'].apply(len)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Magezter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,url,title,title_count,content,content_count,title_tokens,content_tokens,title_tokens_stopwords,title_tokens_stopwords_count,content_tokens_stopwords,content_tokens_stopwords_count
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"ibu, perhatikan 5 hal ini saat memilih car se...",11,"bagi orang tua yang memiliki kendaraan, memili...",575,"[ibu, ,, perhatikan, 5, hal, ini, saat, memili...","[bagi, orang, tua, yang, memiliki, kendaraan, ...","[,, perhatikan, 5, memilih, car, seat, newborn]",7,"[orang, tua, memiliki, kendaraan, ,, memilihca...",339
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"catat, ini 6 tips menyimpan beras agar tidak ...",10,sebagai makanan pokok beras harus disimpan den...,617,"[catat, ,, ini, 6, tips, menyimpan, beras, aga...","[sebagai, makanan, pokok, beras, harus, disimp...","[catat, ,, 6, tips, menyimpan, beras, berkutu]",7,"[makanan, pokok, beras, disimpan, kualitasnya,...",347
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"waspada, ini 6 bahaya asap rokok bagi kesehat...",10,merokok tidak hanya berbahaya untuk dirimu sen...,708,"[waspada, ,, ini, 6, bahaya, asap, rokok, bagi...","[merokok, tidak, hanya, berbahaya, untuk, diri...","[waspada, ,, 6, bahaya, asap, rokok, kesehatan...",8,"[merokok, berbahaya, dirimu, orang-orang, seki...",438
3,https://www.halodoc.com/sering-terkena-polusi-...,"sering terkena polusi, ketahui berbagi cara b...",9,"sehari-hari, manusia terpapar polusi dalam jum...",569,"[sering, terkena, polusi, ,, ketahui, berbagi,...","[sehari-hari, ,, manusia, terpapar, polusi, da...","[terkena, polusi, ,, ketahui, berbagi, bersihk...",7,"[sehari-hari, ,, manusia, terpapar, polusi, as...",342
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,ini risiko dan manfaat implan payudara yang p...,9,implan payudara adalah prosedur bedah kosmetik...,613,"[ini, risiko, dan, manfaat, implan, payudara, ...","[implan, payudara, adalah, prosedur, bedah, ko...","[risiko, manfaat, implan, payudara]",4,"[implan, payudara, prosedur, bedah, kosmetik, ...",362


In [30]:
# Stemming (using Sastrawi)
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stemming title
df['title_tokens_stopwords_stemmed'] = df['title_tokens_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
df['title_tokens_stopwords_stemmed_count'] = df['title_tokens_stopwords_stemmed'].apply(len)

# Stemming content
df['content_tokens_stopwords_stemmed'] = df['content_tokens_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
df['content_tokens_stopwords_stemmed_count'] = df['content_tokens_stopwords_stemmed'].apply(len)

- Stopwords removal takes 0.8s
- Stemming takes 108m 14.7s

In [ ]:
# # save df (for later use)
# df.to_csv('tf-idf_halodocnew_1.csv', index=False)